<a href="https://colab.research.google.com/github/HUJameson/Colab/blob/main/aillm_0209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

Mounted at /content/drive


In [ ]:
!pip install openai
import openai
from sk_utils import read_sk
openai_sk = read_sk()
%env OPENAI_API_KEY=$openai_sk

In [ ]:
!pip install langchain

In [4]:
import openai, os

openai.api_key = os.environ.get("OPENAI_API_KEY")

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAIChat
from langchain.chains import LLMChain

llm = OpenAIChat(max_tokens=2048, temperature=0.5)
multiple_choice = """
请针对 >>> 和 <<< 中间的用户问题，选择一个合适的工具去回答她的问题。只要用A、B、C的选项字母告诉我答案。
如果你觉得都不合适，就选D。

>>>{question}<<<

我们有的工具包括：
A. 一个能够查询商品信息，为用户进行商品导购的工具
B. 一个能够查询订单信息，获得最新的订单情况的工具
C. 一个能够搜索商家的退换货政策、运费、物流时长、支付渠道、覆盖国家的工具
D. 都不合适
"""
multiple_choice_prompt = PromptTemplate(template=multiple_choice, input_variables=["question"])
choice_chain = LLMChain(llm=llm, prompt=multiple_choice_prompt, output_key="answer")

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [5]:
question = "我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？"
print(choice_chain(question))

{'question': '我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？', 'answer': 'A. 一个能够查询商品信息，为用户进行商品导购的工具'}


In [6]:
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
print(choice_chain(question))

{'question': '我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？', 'answer': 'B. 一个能够查询订单信息，获得最新的订单情况的工具'}


In [7]:
question = "请问你们的货，能送到三亚吗？大概需要几天？"
print(choice_chain(question))

{'question': '请问你们的货，能送到三亚吗？大概需要几天？', 'answer': 'C. 一个能够搜索商家的退换货政策、运费、物流时长、支付渠道、覆盖国家的工具'}


In [8]:
question = "今天天气怎么样？"
print(choice_chain(question))

{'question': '今天天气怎么样？', 'answer': 'D. 都不合适'}


In [9]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

def search_order(input: str) -> str:
    return "订单状态：已发货；发货日期：2023-01-01；预计送达时间：2023-01-10"

def recommend_product(input: str) -> str:
    return "红色连衣裙"

def faq(intput: str) -> str:
    return "7天无理由退货"

tools = [
    Tool(
        name = "Search Order",func=search_order,
        description="useful for when you need to answer questions about customers orders"
    ),
    Tool(name="Recommend Product", func=recommend_product,
         description="useful for when you need to answer questions about product recommendations"
    ),
    Tool(name="FAQ", func=faq,
         description="useful for when you need to answer questions about shopping policies, like return policy, shipping policy, etc."
    )
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [10]:
question = "我想买一件衣服，但是不知道哪个款式好看，你能帮我推荐一下吗？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
 I need to recommend a product.
Action: Recommend Product
Action Input: Clothing
Observation: 红色连衣裙
Thought: I now know the final answer.
Final Answer: 我推荐红色连衣裙。

> Finished chain.
我推荐红色连衣裙。


In [11]:
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
 I need to find out the status of this order
Action: Search Order
Action Input: 2022ABCDE
Observation: 订单状态：已发货；发货日期：2023-01-01；预计送达时间：2023-01-10
Thought: I now know the final answer
Final Answer: 您的订单 2022ABCDE 已发货，预计将于2023-01-10送达。

> Finished chain.
您的订单 2022ABCDE 已发货，预计将于2023-01-10送达。


In [12]:
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
 I need to find out the shipping policy and delivery time
Action: FAQ
Action Input: Shipping policy and delivery time
Observation: 7天无理由退货
Thought: I need to find out the delivery time
Action: FAQ
Action Input: Delivery time
Observation: 7天无理由退货
Thought:

 I need to find out if we can deliver to Sanya
Action: FAQ
Action Input: Delivery to Sanya
Observation: 7天无理由退货
Thought:

 I now know the final answer
Final Answer: 我们可以把货送到三亚，大概需要7天。

> Finished chain.
我们可以把货送到三亚，大概需要7天。


In [13]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", max_iterations = 2, verbose=True)
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = agent.run(question)
print("===")
print(result)
print("===")



> Entering new AgentExecutor chain...
 I need to find out the shipping policy and delivery time
Action: FAQ
Action Input: Shipping policy and delivery time
Observation: 7天无理由退货
Thought: I need to find out the delivery time
Action: FAQ
Action Input: Delivery time
Observation: 7天无理由退货
Thought:

> Finished chain.
===
Agent stopped due to iteration limit or time limit.
===


In [ ]:
!pip install spacy
!python -m spacy download zh_core_web_sm
!pip install tiktoken
!pip install faiss-cpu

In [19]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import SpacyTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import TextLoader

llm = OpenAI(temperature=0)
loader = TextLoader('./data/ecommerce/ecommerce_faq.txt')
documents = loader.load()
text_splitter = SpacyTextSplitter(chunk_size=256, pipeline="zh_core_web_sm")
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_documents(texts, embeddings)

faq_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=docsearch, verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:251: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [21]:
from langchain.agents import tool

@tool("FAQ")
def faq(intput: str) -> str:
    """"useful for when you need to answer questions about shopping policies, like return policy, shipping policy, etc."""
    return faq_chain.run(intput)

tools = [
    Tool(
        name = "Search Order",func=search_order,
        description="useful for when you need to answer questions about customers orders"
    ),
    Tool(name="Recommend Product", func=recommend_product,
         description="useful for when you need to answer questions about product recommendations"
    ),
    faq
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [22]:
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = agent.run(question)
print(result)



> Entering new AgentExecutor chain...
 I need to find out the shipping policy and delivery time.
Action: FAQ
Action Input: shipping policy and delivery time

> Entering new VectorDBQA chain...

> Finished chain.

Observation:  我们支持全国大部分省份的配送，一般情况下，大部分城市的订单在2-3个工作日内送达，偏远地区可能需要5-7个工作日。具体送货时间可能因订单商品、配送地址和物流公司而异。
Thought: I now know the final answer
Final Answer: 我们支持全国大部分省份的配送，一般情况下，大部分城市的订单在2-3个工作日内送达，偏远地区可能需要5-7个工作日。具体送货时间可能因订单商品、配送地址和物流公司而异。

> Finished chain.
我们支持全国大部分省份的配送，一般情况下，大部分城市的订单在2-3个工作日内送达，偏远地区可能需要5-7个工作日。具体送货时间可能因订单商品、配送地址和物流公司而异。


In [23]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import CSVLoader

product_loader = CSVLoader('./data/ecommerce/ecommerce_products.csv')
product_documents = product_loader.load()
product_text_splitter = CharacterTextSplitter(chunk_size=1024, separator="\n")
product_texts = product_text_splitter.split_documents(product_documents)
product_search = FAISS.from_documents(product_texts, OpenAIEmbeddings())
product_chain = VectorDBQA.from_chain_type(llm=llm, vectorstore=product_search, verbose=True)

@tool("FAQ")
def faq(intput: str) -> str:
    """"useful for when you need to answer questions about shopping policies, like return policy, shipping policy, etc."""
    return faq_chain.run(input)

@tool("Recommend Product")
def recommend_product(input: str) -> str:
    """"useful for when you need to search and recommend products and recommend it to the user"""
    return product_chain.run(input)

tools = [
    Tool(
        name = "Search Order",func=search_order,
        description="useful for when you need to answer questions about customers orders"
    ),
    recommend_product, faq]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [24]:
question = "我想买一件衣服，想要在春天去公园穿，但是不知道哪个款式好看，你能帮我推荐一下吗？"
answer = agent.run(question)
print(answer)



> Entering new AgentExecutor chain...
 I need to recommend a product to the user.
Action: Recommend Product
Action Input: Clothing for park in spring

> Entering new VectorDBQA chain...

> Finished chain.

Observation:  休闲、简约风格的衣服，比如长款风衣、卫衣连衣裙、长款卫衣等。
Thought: I now know the final answer.
Final Answer: 我建议您可以选择休闲、简约风格的衣服，比如长款风衣、卫衣连衣裙、长款卫衣等。

> Finished chain.
我建议您可以选择休闲、简约风格的衣服，比如长款风衣、卫衣连衣裙、长款卫衣等。


In [27]:
import json

ORDER_1 = "20230101ABC"
ORDER_2 = "20230101EFG"

ORDER_1_DETAIL = {
    "order_number": ORDER_1,
    "status": "已发货",
    "shipping_date" : "2023-01-03",
    "estimated_delivered_date": "2023-01-05",
}

ORDER_2_DETAIL = {
    "order_number": ORDER_2,
    "status": "未发货",
    "shipping_date" : None,
    "estimated_delivered_date": None,
}

import re

@tool("Search Order")
def search_order(input:str)->str:
    """一个帮助用户查询最新订单状态的工具，并且能处理以下情况：
    1. 在用户没有输入订单号的时候，会询问用户订单号
    2. 在用户输入的订单号查询不到的时候，会让用户二次确认订单号是否正确"""
    pattern = r"\d+[A-Z]+"
    match = re.search(pattern, input)

    order_number = input
    if match:
        order_number = match.group(0)
    else:
        return "请问您的订单号是多少？"
    if order_number == ORDER_1:
        return json.dumps(ORDER_1_DETAIL)
    elif order_number == ORDER_2:
        return json.dumps(ORDER_2_DETAIL)
    else:
        return f"对不起，根据{input}没有找到您的订单"

tools = [search_order,recommend_product, faq]
agent = initialize_agent(tools, llm=OpenAI(temperature=0), agent="zero-shot-react-description", verbose=True)

In [28]:
question = "我有一张订单，订单号是 2022ABCDE，一直没有收到，能麻烦帮我查一下吗？"
answer = agent.run(question)
print(answer)



> Entering new AgentExecutor chain...
 我需要查询订单状态
Action: Search Order
Action Input: 2022ABCDE
Observation: 对不起，根据2022ABCDE没有找到您的订单
Thought: 我需要再次确认订单号是否正确
Action: Search Order
Action Input: 2022ABCDE
Observation: 对不起，根据2022ABCDE没有找到您的订单
Thought: 我现在知道最终答案
Final Answer: 对不起，根据您提供的订单号2022ABCDE没有找到您的订单，请您再次确认订单号是否正确。

> Finished chain.
对不起，根据您提供的订单号2022ABCDE没有找到您的订单，请您再次确认订单号是否正确。


In [30]:
import json
from langchain.chat_models import ChatOpenAI

ORDER_1 = "20230101ABC"
ORDER_2 = "20230101EFG"

ORDER_1_DETAIL = {
    "order_number": ORDER_1,
    "status": "已发货",
    "shipping_date" : "2023-01-03",
    "estimated_delivered_date": "2023-01-05",
}

ORDER_2_DETAIL = {
    "order_number": ORDER_2,
    "status": "未发货",
    "shipping_date" : None,
    "estimated_delivered_date": None,
}

import re

answer_order_info = PromptTemplate(
    template="请把下面的订单信息回复给用户： \n\n {order}?", input_variables=["order"]
)
answer_order_llm = LLMChain(llm=ChatOpenAI(temperature=0),  prompt=answer_order_info)

@tool("Search Order", return_direct=True)
def search_order(input:str)->str:
    """useful for when you need to answer questions about customers orders"""
    pattern = r"\d+[A-Z]+"
    match = re.search(pattern, input)

    order_number = input
    if match:
        order_number = match.group(0)
    else:
        return "请问您的订单号是多少？"
    if order_number == ORDER_1:
        return answer_order_llm.run(json.dumps(ORDER_1_DETAIL))
    elif order_number == ORDER_2:
        return answer_order_llm.run(json.dumps(ORDER_2_DETAIL))
    else:
        return f"对不起，根据{input}没有找到您的订单"

from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

tools = [search_order,recommend_product, faq]
chatllm=ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation_agent = initialize_agent(tools, chatllm,
                                      agent="conversational-react-description",
                                      memory=memory, verbose=True)

In [31]:
question1 = "我有一张订单，一直没有收到，能麻烦帮我查一下吗？"
answer1 = conversation_agent.run(question1)
print(answer1)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search Order
Action Input: 我有一张订单，一直没有收到，能麻烦帮我查一下吗？
Observation: 请问您的订单号是多少？


> Finished chain.
请问您的订单号是多少？


In [32]:
question2 = "我的订单号是20230101ABC"
answer2 = conversation_agent.run(question2)
print(answer2)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search Order
Action Input: 20230101ABC
Observation: 尊敬的用户，以下是您的订单信息：

订单编号：20230101ABC
订单状态：已发货
发货日期：2023年1月3日
预计送达日期：2023年1月5日

如有任何问题，请随时与我们联系。感谢您的支持！


> Finished chain.
尊敬的用户，以下是您的订单信息：

订单编号：20230101ABC
订单状态：已发货
发货日期：2023年1月3日
预计送达日期：2023年1月5日

如有任何问题，请随时与我们联系。感谢您的支持！


In [34]:
question3 = "你们的退货政策是怎么样的？"
answer3 = conversation_agent.run(question3)
print(answer3)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: FAQ
Action Input: 退货政策

> Entering new VectorDBQA chain...

> Finished chain.

Observation:  自收到商品之日起7天内，如产品未使用、包装完好，您可以申请退货。某些特殊商品可能不支持退货，请在购买前查看商品详情页面的退货政策。
Thought:

OutputParserException: ignored